In [1]:
import pandas as pd
import numpy as np
import os
#Mac
os.chdir('/Users/yaircardenasavila/Desktop/Github_repo')
#Windows
#os.chdir(r'C:\Users\armando_cardenas\Desktop\Github_repo\HH_Index')

<span style = "color:red">2026</span>

In [15]:
cols_2026 = ['Clave Unidad Presupuestal', 'Unidad Presupuestal', 'Clave Unidad Responsable', 'Unidad Responsable', 'Clave Programa Presupuestario', 'Programa Presupuestario', 
        'Clave Componente', 'Componente', 'Clave Destino del Gasto Específico', 'Destino del Gasto Específico',
        'Clave Fuente de Financiamiento', 'Fuente de Financiamiento', 'Techo Comunicado', 'Monto Determinado', 'Adicional']
base_2026 = pd.read_excel('HH_Index/Fases de anteproyecto 2026 actualización de componentes transversalidad (1).xlsx', skiprows=1, usecols=cols_2026,
                          dtype={'Clave Unidad Presupuestal':str,
                                'Unidad Presupuestal':str,
                                'Clave Unidad Responsable':str,
                                'Unidad Responsable':str,
                                'Clave Programa Presupuestario':str,
                                'Programa Presupuestario':str,
                                'Clave Componente':str,
                                'Componente':str,
                                'Clave Destino del Gasto Específico':str,
                                'Destino del Gasto Específico':str,
                                'Clave Fuente de Financiamiento':str,
                                'Fuente de Financiamiento':str,
                                'Techo Comunicado':float,
                                'Monto Determinado':float,
                                'Adicional':float})

base_2026 = base_2026.rename(columns={'Concepto Capitulo 2':' NOMBRE Capitulo 2',
                              'Capitulo 2': 'CLAVE Capitulo 2'})
base_2026['llave'] = base_2026['Clave Unidad Presupuestal'] + '_' + base_2026['Clave Unidad Responsable']

In [55]:
#Base de información para 2026
info_2026 = base_2026[['llave', 'Unidad Presupuestal', 'Unidad Responsable', 'Clave Programa Presupuestario', 'Programa Presupuestario', 'Clave Componente', 'Componente']].drop_duplicates()

<span style = "color:yellow">Análisis de cada PP con respecto a su UR</span>

In [45]:
##Peso porcentual de cada programa presupuestario con respecto al total distribuido de la UR
PP_UR_2026 = base_2026.groupby(by=['llave', 'Clave Programa Presupuestario']).agg({'Monto Determinado':'sum', 'Adicional':'sum'}).reset_index()
PP_UR_2026['prop'] = (
    PP_UR_2026['Monto Determinado'] /
    PP_UR_2026.groupby(by=['llave'])['Monto Determinado'].transform('sum') 
)
PP_UR_2026 = pd.merge(left=PP_UR_2026 ,right=info_2026[['llave', 'Clave Programa Presupuestario', 'Unidad Presupuestal', 'Unidad Responsable', 'Programa Presupuestario']], on=['llave', 'Clave Programa Presupuestario'], how='left').drop_duplicates(subset='Clave Programa Presupuestario')
PP_UR_2026

,llave,Clave Programa Presupuestario,Monto Determinado,Adicional,prop,Unidad Presupuestal,Unidad Responsable,Programa Presupuestario
0,01_000,A01,7790003.0,0.0,1.000000,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador
2,02_000,A02,528767891.0,254071771.0,0.983614,Secretaría General de Gobierno,Secretaría General de Gobierno,Gestión de la Secretaría General de Gobierno
29,02_000,SG1,8808588.0,1150000.0,0.016386,Secretaría General de Gobierno,Secretaría General de Gobierno,Ejercicio pleno y equitativo de los derechos h...
46,02_117,SPC,321151271.0,13412000.0,1.000000,Secretaría General de Gobierno,Unidad Estatal de Protección Civil y Bomberos,Gestión Integral de Riesgos y Protección Civil
52,02_172,SJ7,100000000.0,0.0,1.000000,Secretaría General de Gobierno,Comisión Estatal Indígena,Desarrollo integral e intercultural de los pue...
...,...,...,...,...,...,...,...,...
1027,52_000,SJ9,37939460.0,2300540.0,1.000000,Instituto de Información Estadística y Geográf...,Instituto de Información Estadística y Geográf...,Decisiones Estratégicas con Información Estrat...
1034,53_000,S53,704371249.0,0.0,1.000000,Instituto de la Infraestructura Física Educati...,Instituto de la Infraestructura Física Educati...,Operación de Programas de Infraestructura Escolar
1039,54_000,S5V,197963000.0,664000000.0,1.000000,Instituto Jalisciense de la Vivienda,Instituto Jalisciense de la Vivienda,Acceso a la Vivienda Adecuada en Jalisco
1044,57_000,SRT,75553197.0,42627081.0,1.000000,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión


$$
HHI^N = \frac{HHI - \tfrac{1}{n}}{1 - \tfrac{1}{n}}
$$

In [46]:
#Indicador de que tan concentrado se encuentra el recurso de una UR en sus programas presupueatarios
#todos los OPDs en su mayoría deben ser 1
# Indice HH
PP_UR_HHI_N_2026 = (
    PP_UR_2026.groupby(by="llave")["prop"]
    .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
).reset_index().fillna(1)
PP_UR_HHI_N_2026 = pd.merge(left=PP_UR_HHI_N_2026, right=info_2026[['llave', 'Unidad Presupuestal', 'Unidad Responsable']].drop_duplicates(subset='llave'), on='llave', how='left').drop_duplicates(subset='llave')
PP_UR_HHI_N_2026

/var/folders/0n/rxsy4y5152719y5rcb1t_dk80000gn/T/ipykernel_10193/1070702346.py:4: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')


,llave,HHI,Unidad Presupuestal,Unidad Responsable
0,01_000,1.000000,Despacho del Gobernador,Despacho del Gobernador
1,02_000,0.935531,Secretaría General de Gobierno,Secretaría General de Gobierno
2,02_117,1.000000,Secretaría General de Gobierno,Unidad Estatal de Protección Civil y Bomberos
3,02_172,1.000000,Secretaría General de Gobierno,Comisión Estatal Indígena
4,02_173,1.000000,Secretaría General de Gobierno,Fideicomiso Atención a Jaliscienses en el Extr...
...,...,...,...,...
130,52_000,1.000000,Instituto de Información Estadística y Geográf...,Instituto de Información Estadística y Geográf...
131,53_000,1.000000,Instituto de la Infraestructura Física Educati...,Instituto de la Infraestructura Física Educati...
132,54_000,1.000000,Instituto Jalisciense de la Vivienda,Instituto Jalisciense de la Vivienda
133,57_000,1.000000,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión


<span style = "color:yellow">Análisis de cada comp con respecro a su PP</span>

In [48]:
#Peso porcentual de cada componente con el total de su programa presupuestario
COMP_UR_2026 = base_2026.groupby(by=['llave', 'Clave Programa Presupuestario', 'Clave Componente']).agg({'Monto Determinado':'sum'}).reset_index()
COMP_UR_2026['prop'] = (
    COMP_UR_2026['Monto Determinado'] /
    COMP_UR_2026.groupby(by=['llave', 'Clave Programa Presupuestario'])['Monto Determinado'].transform('sum')
)
COMP_UR_2026 = pd.merge(left=COMP_UR_2026 ,right=info_2026[['llave', 'Clave Programa Presupuestario', 'Clave Componente', 'Unidad Presupuestal', 'Unidad Responsable', 'Programa Presupuestario', 'Componente']], on=['llave', 'Clave Programa Presupuestario', 'Clave Componente'], how='left').drop_duplicates(subset=['llave', 'Clave Programa Presupuestario', 'Clave Componente'])
COMP_UR_2026

,llave,Clave Programa Presupuestario,Clave Componente,Monto Determinado,prop,Unidad Presupuestal,Unidad Responsable,Programa Presupuestario,Componente
0,01_000,A01,01,7500052.0,0.962779,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador,Gastos básicos para la operación
1,01_000,A01,02,289951.0,0.037221,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador,Atención Integral del Despacho del C. Gobernador
2,02_000,A02,04,435155.0,0.000823,Secretaría General de Gobierno,Secretaría General de Gobierno,Gestión de la Secretaría General de Gobierno,"Garantizar la autenticidad, validez legal y tr..."
3,02_000,A02,05,6582270.0,0.012448,Secretaría General de Gobierno,Secretaría General de Gobierno,Gestión de la Secretaría General de Gobierno,Ejercer y coordinar la función registral en ma...
4,02_000,A02,08,108330.0,0.000205,Secretaría General de Gobierno,Secretaría General de Gobierno,Gestión de la Secretaría General de Gobierno,Atender con eficiencia y enfoque ciudadano las...
...,...,...,...,...,...,...,...,...,...
1044,57_000,SRT,A1,4567144.0,0.060449,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión,Contenidos culturales y educativos transmitido...
1045,57_000,SRT,A2,51535935.0,0.682115,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión,Gestión de los recursos destinados a la operac...
1046,57_000,SRT,A3,600050.0,0.007942,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión,"Programas informativos, educativos y culturale..."
1047,57_000,SRT,A4,18850068.0,0.249494,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión,Señal de television digital terrestre transmid...


$$
HHI^N = \frac{HHI - \tfrac{1}{n}}{1 - \tfrac{1}{n}}
$$

In [50]:
#Indicador de que tan concentrado se encuentra el dinero entre los componentes de un programa presupuestario
COMO_UR_HHI_N_2026 = (
    COMP_UR_2026.groupby(by=['llave', 'Clave Programa Presupuestario'])["prop"]
    .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
).reset_index().fillna(1)
COMO_UR_HHI_N_2026 = pd.merge(left=COMO_UR_HHI_N_2026, right=info_2026[['llave', 'Clave Programa Presupuestario', 'Unidad Presupuestal', 'Unidad Responsable', 'Programa Presupuestario']].drop_duplicates(subset=['llave', 'Clave Programa Presupuestario']), on=['llave', 'Clave Programa Presupuestario'], how='left').drop_duplicates(subset=['llave', 'Clave Programa Presupuestario'])
COMO_UR_HHI_N_2026

/var/folders/0n/rxsy4y5152719y5rcb1t_dk80000gn/T/ipykernel_10193/1477800388.py:3: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')


,llave,Clave Programa Presupuestario,HHI,Unidad Presupuestal,Unidad Responsable,Programa Presupuestario
0,01_000,A01,0.856658,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador
1,02_000,A02,0.682402,Secretaría General de Gobierno,Secretaría General de Gobierno,Gestión de la Secretaría General de Gobierno
2,02_000,SG1,0.339741,Secretaría General de Gobierno,Secretaría General de Gobierno,Ejercicio pleno y equitativo de los derechos h...
3,02_117,SPC,0.445289,Secretaría General de Gobierno,Unidad Estatal de Protección Civil y Bomberos,Gestión Integral de Riesgos y Protección Civil
4,02_172,SJ7,0.369771,Secretaría General de Gobierno,Comisión Estatal Indígena,Desarrollo integral e intercultural de los pue...
...,...,...,...,...,...,...
225,52_000,SJ9,0.068930,Instituto de Información Estadística y Geográf...,Instituto de Información Estadística y Geográf...,Decisiones Estratégicas con Información Estrat...
226,53_000,S53,0.244733,Instituto de la Infraestructura Física Educati...,Instituto de la Infraestructura Física Educati...,Operación de Programas de Infraestructura Escolar
227,54_000,S5V,0.120140,Instituto Jalisciense de la Vivienda,Instituto Jalisciense de la Vivienda,Acceso a la Vivienda Adecuada en Jalisco
228,57_000,SRT,0.374993,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión


<span style = "color:red">2025</span>

In [58]:
cols_2025 = ['Clave Unidad Presupuestal', 'Unidad Presupuestal', 'Clave Unidad Responsable', 'Unidad Responsable', 'Clave Programa Presupuestario', 'Programa Presupuestario', 
             'Clave Componente', 'Componente', 'Clave Destino del Gasto Específico', 'Destino del Gasto Específico',
             'Clave Fuente de Financiamiento', 'Fuente de Financiamiento', 'Techo Comunicado', 'Monto Determinado', 'Adicional']

base_2025 = pd.read_excel('HH_Index/Fases de anteproyecto 2025.xlsx', sheet_name='Fases', usecols=cols_2025,
                          dtype={'Clave Unidad Presupuestal':str,
                                'Unidad Presupuestal':str,
                                'Clave Unidad Responsable':str,
                                'Unidad Responsable':str,
                                'Clave Programa Presupuestario':str,
                                'Programa Presupuestario':str,
                                'Clave Componente':str,
                                'Componente':str,
                                'Clave Destino del Gasto Específico':str,
                                'Destino del Gasto Específico':str,
                                'Clave Fuente de Financiamiento':str,
                                'Fuente de Financiamiento':str,
                                'Techo Comunicado':float,
                                'Monto Determinado':float,
                                'Adicional':float})

base_2025['llave'] = base_2025['Clave Unidad Presupuestal'] + '_' + base_2025['Clave Unidad Responsable']

In [56]:
info_2025 = base_2025[['llave', 'Unidad Presupuestal', 'Unidad Responsable', 'Clave Programa Presupuestario', 'Programa Presupuestario', 'Clave Componente', 'Componente']].drop_duplicates()

<span style = "color:yellow">Análisis de cada PP con respecto a su UR</span>

In [51]:
##Peso porcentual de cada programa presupuestario con respecto al total distribuido de la UR
PP_UR_2025 = base_2025.groupby(by=['llave', 'Clave Programa Presupuestario']).agg({'Monto Determinado':'sum', 'Adicional':'sum'}).reset_index()
PP_UR_2025['prop'] = (
    PP_UR_2025['Monto Determinado'] /
    PP_UR_2025.groupby(by=['llave'])['Monto Determinado'].transform('sum') 
)
PP_UR_2025 = pd.merge(left=PP_UR_2025 ,right=info_2025[['llave', 'Clave Programa Presupuestario', 'Unidad Presupuestal', 'Unidad Responsable', 'Programa Presupuestario']], on=['llave', 'Clave Programa Presupuestario'], how='left').drop_duplicates(subset='Clave Programa Presupuestario')
PP_UR_2025

PP_UR_2025

,llave,Clave Programa Presupuestario,Monto Determinado,Adicional,prop,Unidad Presupuestal,Unidad Responsable,Programa Presupuestario
0,01_000,636,7642174.0,0.0,1.000000,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador
1,02_000,010,4707743.0,0.0,0.008887,Secretaría General de Gobierno,Secretaría General de Gobierno,"Gobierno digital en línea, innovación y crecim..."
6,02_000,015,40775468.0,0.0,0.076973,Secretaría General de Gobierno,Secretaría General de Gobierno,Modernización del Registro Civil
11,02_000,019,117035674.0,0.0,0.220932,Secretaría General de Gobierno,Secretaría General de Gobierno,Atención y seguimiento a los recursos financie...
16,02_000,026,28265265.0,0.0,0.053357,Secretaría General de Gobierno,Secretaría General de Gobierno,Atención a los Asuntos y Conflictos Agrarios e...
...,...,...,...,...,...,...,...,...
1498,48_000,588,10670132.0,0.0,1.000000,Fiscalía Especializada en Materia de Delitos E...,Fiscalía Especializada en Materia de Delitos E...,Investigar y Perseguir los Delitos Electorales...
1503,49_000,670,65264240.0,0.0,1.000000,Secretaría de Inteligencía y Busqueda de Personas,Secretaría de Inteligencía y Busqueda de Personas,Mecanismos de Inteligencia y Coordinación para...
1507,50_000,848,20049515.0,0.0,1.000000,Archivo General del Estado,Archivo General del Estado,Coordinación oportuna de la gestión documental...
1509,51_000,829,7476240.0,0.0,1.000000,Secretaría de Desarrollo Energético Sustentable,Secretaría de Desarrollo Energético Sustentable,Administración y gestión energética del estado...


$$
HHI^N = \frac{HHI - \tfrac{1}{n}}{1 - \tfrac{1}{n}}
$$

In [52]:
#Indicador de que tan concentrado se encuentra el recurso de una UR en sus programas presupueatarios
#Indice HH
PP_UR_HHI_N_2025 = (
    PP_UR_2025.groupby(by="llave")["prop"]
    .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
).reset_index().fillna(1)

PP_UR_HHI_N_2025 = pd.merge(left=PP_UR_HHI_N_2025, right=info_2025[['llave', 'Unidad Presupuestal', 'Unidad Responsable']].drop_duplicates(subset='llave'), on='llave', how='left').drop_duplicates(subset='llave')
PP_UR_HHI_N_2025

PP_UR_HHI_N_2025

/var/folders/0n/rxsy4y5152719y5rcb1t_dk80000gn/T/ipykernel_10193/3612855974.py:4: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
/var/folders/0n/rxsy4y5152719y5rcb1t_dk80000gn/T/ipykernel_10193/3612855974.py:4: RuntimeWarning: divide by zero encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')


,llave,HHI,Unidad Presupuestal,Unidad Responsable
0,01_000,1.000000,Despacho del Gobernador,Despacho del Gobernador
1,02_000,0.108329,Secretaría General de Gobierno,Secretaría General de Gobierno
2,02_117,0.755522,Secretaría General de Gobierno,Unidad Estatal de Protección Civil y Bomberos
3,02_172,1.000000,Secretaría General de Gobierno,Comisión Estatal Indígena
4,02_173,1.000000,Secretaría General de Gobierno,Fideicomiso Atención a Jaliscienses en el Extr...
...,...,...,...,...
127,48_000,1.000000,Fiscalía Especializada en Materia de Delitos E...,Fiscalía Especializada en Materia de Delitos E...
128,49_000,1.000000,Secretaría de Inteligencía y Busqueda de Personas,Secretaría de Inteligencía y Busqueda de Personas
129,50_000,1.000000,Archivo General del Estado,Archivo General del Estado
130,51_000,1.000000,Secretaría de Desarrollo Energético Sustentable,Secretaría de Desarrollo Energético Sustentable


<span style = "color:yellow">Análisis de cada comp con respecro a su PP</span>

In [53]:
#Peso porcentual que tiene cada componente con respecto al total presupuestado de su PP
COMP_UR_2025 = base_2025.groupby(by=['llave', 'Clave Programa Presupuestario', 'Clave Componente']).agg({'Monto Determinado':'sum'}).reset_index()
COMP_UR_2025['prop'] = (
    COMP_UR_2025['Monto Determinado'] /
    COMP_UR_2025.groupby(by=['llave', 'Clave Programa Presupuestario'])['Monto Determinado'].transform('sum')
)
COMP_UR_2025 = pd.merge(left=COMP_UR_2025 ,right=info_2025[['llave', 'Clave Programa Presupuestario', 'Clave Componente', 'Unidad Presupuestal', 'Unidad Responsable', 'Programa Presupuestario', 'Componente']], on=['llave', 'Clave Programa Presupuestario', 'Clave Componente'], how='left').drop_duplicates(subset=['llave', 'Clave Programa Presupuestario', 'Clave Componente'])
COMP_UR_2025

,llave,Clave Programa Presupuestario,Clave Componente,Monto Determinado,prop,Unidad Presupuestal,Unidad Responsable,Programa Presupuestario,Componente
0,01_000,636,C1,7642174.0,1.000000,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador,"Ciudadanos y asuntos atendidos, orientados y/o..."
1,02_000,010,C1,1597999.0,0.339441,Secretaría General de Gobierno,Secretaría General de Gobierno,"Gobierno digital en línea, innovación y crecim...",Cédulas expedidas a fin que los profesionistas...
2,02_000,010,C2,796806.0,0.169254,Secretaría General de Gobierno,Secretaría General de Gobierno,"Gobierno digital en línea, innovación y crecim...",Servicio de expedición de cédulas brindado y m...
3,02_000,010,C3,386330.0,0.082063,Secretaría General de Gobierno,Secretaría General de Gobierno,"Gobierno digital en línea, innovación y crecim...",Campaña de difusión realizada para la sociedad...
4,02_000,010,C4,445057.0,0.094537,Secretaría General de Gobierno,Secretaría General de Gobierno,"Gobierno digital en línea, innovación y crecim...",Reconocimiento realizado en acto público presi...
...,...,...,...,...,...,...,...,...,...
1526,54_000,363,A4,45000.0,0.000278,Instituto Jalisciense de la Vivienda,Instituto Jalisciense de la Vivienda,Fortalecimiento de la política pública de vivi...,Proyectos estratégicos realizados de vivienda
1527,57_000,436,D1,55713488.0,0.737407,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión Jalisciense,Señal de televisión digital terrestre transmit...
1528,57_000,436,D4,10957759.0,0.145034,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión Jalisciense,"Contenidos informativos, culturales y educativ..."
1529,57_000,436,D5,8881950.0,0.117559,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión Jalisciense,Programas de radio transmitidos por Jalisco Radio


$$
HHI^N = \frac{HHI - \tfrac{1}{n}}{1 - \tfrac{1}{n}}
$$

In [54]:
#Indicador de que tan concentrado se encuentra el recurso entre los componentes de un PP
COMO_UR_HHI_N_2025 = (
    COMP_UR_2025.groupby(by=['llave', 'Clave Programa Presupuestario'])["prop"]
    .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
).reset_index().fillna(1)

COMO_UR_HHI_N_2025 = pd.merge(left=COMO_UR_HHI_N_2025, right=info_2025[['llave', 'Clave Programa Presupuestario', 'Unidad Presupuestal', 'Unidad Responsable', 'Programa Presupuestario']].drop_duplicates(subset=['llave', 'Clave Programa Presupuestario']), on=['llave', 'Clave Programa Presupuestario'], how='left').drop_duplicates(subset=['llave', 'Clave Programa Presupuestario'])


COMO_UR_HHI_N_2025

/var/folders/0n/rxsy4y5152719y5rcb1t_dk80000gn/T/ipykernel_10193/3161078060.py:3: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
/var/folders/0n/rxsy4y5152719y5rcb1t_dk80000gn/T/ipykernel_10193/3161078060.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')


,llave,Clave Programa Presupuestario,HHI,Unidad Presupuestal,Unidad Responsable,Programa Presupuestario
0,01_000,636,1.000000,Despacho del Gobernador,Despacho del Gobernador,Atención Integral del Despacho del C. Gobernador
1,02_000,010,0.073222,Secretaría General de Gobierno,Secretaría General de Gobierno,"Gobierno digital en línea, innovación y crecim..."
2,02_000,015,0.223945,Secretaría General de Gobierno,Secretaría General de Gobierno,Modernización del Registro Civil
3,02_000,019,0.101366,Secretaría General de Gobierno,Secretaría General de Gobierno,Atención y seguimiento a los recursos financie...
4,02_000,026,0.002636,Secretaría General de Gobierno,Secretaría General de Gobierno,Atención a los Asuntos y Conflictos Agrarios e...
...,...,...,...,...,...,...
445,53_000,812,0.278849,Instituto de la Infraestructura Física Educati...,Instituto de la Infraestructura Física Educati...,Operación de los programas de infraestructura ...
446,53_000,817,0.994235,Instituto de la Infraestructura Física Educati...,Instituto de la Infraestructura Física Educati...,Administración del gasto de operación
447,54_000,363,0.998888,Instituto Jalisciense de la Vivienda,Instituto Jalisciense de la Vivienda,Fortalecimiento de la política pública de vivi...
448,57_000,436,0.367937,Sistema Jalisciense de Radio y Televisión,Sistema Jalisciense de Radio y Televisión,Radio y Televisión Jalisciense


<span style = "color:green">Comparación 2026 vs 2025</span>